Step1: Importing the libraries required throughout the notebook

In [ ]:
%pip install geopy 
%pip install Nominatim
%pip install haversine

  Created wheel for Nominatim: filename=nominatim-0.1-py3-none-any.whl size=2365 sha256=12d3aa2e7ed7735697da08749ae5f1f1fe2df617f3d150360c295689f1adb2bf
  Stored in directory: /root/.cache/pip/wheels/37/00/9e/d904c390bfb174830ad3dcfd62af5544cee7d88bed4f8acedd
Successfully built Nominatim


In [ ]:
!pip install anvil-uplink

     |████████████████████████████████| 64 kB 2.3 MB/s 
     |████████████████████████████████| 51 kB 204 kB/s 
  Created wheel for ws4py: filename=ws4py-0.5.1-py3-none-any.whl size=45229 sha256=69deeeaa1977267db0c0e1f6dc7caca5a52628a4ff8f05269364bc9c56b05951
  Stored in directory: /root/.cache/pip/wheels/29/ea/7d/3410aa0aa0e4402ead9a7a97ab2214804887e0f5c2b76f0c96
Successfully built ws4py


In [ ]:
import anvil.server

In [ ]:
anvil.server.connect("72WGMNOW2CQWI5ODXYVST2MR-PVC34QFBA4AKVZ2Y-CLIENT")

In [ ]:
@anvil.server.callable
def predict_iris(sepal_length, sepal_width, petal_length, petal_width):
  classification = knn.predict([[sepal_length, sepal_width, petal_length, petal_width]])
  return iris.target_names[classification][0]

In [ ]:
## Import library
import numpy as np
from numpy import mean
import pandas as pd
from pandas_profiling import ProfileReport
import seaborn as sns
import warnings
warnings.filterwarnings("ignore")
import matplotlib.pyplot as plt
import matplotlib.colors as colors
%matplotlib inline
from operator import itemgetter
import ast
from ast import literal_eval

## Plotly Packages
from plotly import tools
import plotly.graph_objs as go
import plotly.express as px
from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot
init_notebook_mode(connected=True)
import ipywidgets
from IPython.display import HTML

import scipy.stats as stats

Step2: Connecting to Google Drive, to extract the file we created in Module 1, which has all the trails we recommended

In [ ]:
!pip install -U -q PyDrive
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials
# Authenticate and create the PyDrive client.
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

#### Below we are loading the file we created in module 1. It has data from all trails and the weather information and google ratings we extracted from the APIS.

In [ ]:
link="https://drive.google.com/file/d/1frwvitynwMJMK_dmx5ZF0K-MjHdue8f4/view?usp=sharing"
id="1frwvitynwMJMK_dmx5ZF0K-MjHdue8f4"
downloaded = drive.CreateFile({'id':id}) 
downloaded.GetContentFile('feature_data_allTrails.csv')
all_trails= pd.read_csv('feature_data_allTrails.csv')
all_trails.head()

,Unnamed: 0,trail_id,name,area_name,city_name,state_name,country_name,popularity,length,elevation_gain,...,activity_cross-country-skiing,activity_fly-fishing,activity_paddle-sports,activity_skiing,activity_bike-touring,activity_whitewater-kayaking,activity_rails-trails,activity_ice-climbing,activity_surfing,activity_snowboarding
0,0,10020048,Harding Ice Field Trail,Kenai Fjords National Park,Seward,Alaska,United States,24.8931,15610.598,1161.8976,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,1,10236086,Mount Healy Overlook Trail,Denali National Park,Denali National Park,Alaska,United States,18.0311,6920.162,507.7968,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,2,10267857,Exit Glacier Trail,Kenai Fjords National Park,Seward,Alaska,United States,17.7821,2896.812,81.9912,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,3,10236076,Horseshoe Lake Trail,Denali National Park,Denali National Park,Alaska,United States,16.2674,3379.614,119.7864,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,4,10236082,Triple Lakes Trail,Denali National Park,Denali National Park,Alaska,United States,12.5935,29772.790,1124.7120,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


Step 3: Data Processing for clustering

In [ ]:
print(all_trails['condition'].value_counts())
print(all_trails['state_name'].value_counts())
print(all_trails['city_name'].value_counts())


Clear                   2131
Partly cloudy            505
Heavy snow               155
Light rain               126
Fog                       97
Overcast                  95
Patchy snow possible      52
Mist                      50
Light drizzle             20
Patchy light snow         17
Cloudy                    16
Freezing fog              11
Patchy rain possible      10
Sunny                      8
Patchy heavy snow          7
Light snow                 7
Moderate rain              4
Light sleet                2
Name: condition, dtype: int64
California        707
Washington        320
Wyoming           311
Utah              292
Colorado          262
Virginia          191
Maine             179
Tennessee         175
Arizona           174
Montana           151
North Carolina    118
Texas              94
Ohio               50
Hawaii             49
Florida            31
Alaska             29
Kentucky           28
Nevada             25
North Dakota       23
Oregon             22
South Da

### To make it easier to use, we are converting the state name, city name and the weather conditions into category codes.

In [ ]:
all_trails["state_name_cat"] = all_trails["state_name"].astype('category').cat.codes
all_trails["city_name_cat"] = all_trails["city_name"].astype('category').cat.codes
all_trails["condition_cat"] = all_trails["condition"].astype('category').cat.codes
all_trails.head()

,Unnamed: 0,trail_id,name,area_name,city_name,state_name,country_name,popularity,length,elevation_gain,...,activity_skiing,activity_bike-touring,activity_whitewater-kayaking,activity_rails-trails,activity_ice-climbing,activity_surfing,activity_snowboarding,state_name_cat,city_name_cat,condition_cat
0,0,10020048,Harding Ice Field Trail,Kenai Fjords National Park,Seward,Alaska,United States,24.8931,15610.598,1161.8976,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,225,0
1,1,10236086,Mount Healy Overlook Trail,Denali National Park,Denali National Park,Alaska,United States,18.0311,6920.162,507.7968,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,68,14
2,2,10267857,Exit Glacier Trail,Kenai Fjords National Park,Seward,Alaska,United States,17.7821,2896.812,81.9912,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,225,0
3,3,10236076,Horseshoe Lake Trail,Denali National Park,Denali National Park,Alaska,United States,16.2674,3379.614,119.7864,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,68,14
4,4,10236082,Triple Lakes Trail,Denali National Park,Denali National Park,Alaska,United States,12.5935,29772.790,1124.7120,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,68,14


Step 4: Data grouping by difficulty
#### As the first part of the project, we are grouping the data by difficulty rating. On the UI we will be asking the user to input their difficulty level. We will be providing recommendations which are below or eqaul to the difficulty level they provide. For example, if the user indicates that they are comfortable with moderate trails, the engine will consider all the trails which are easy or moderate and will skip the difficult and strenous trails

In [ ]:
data_by_difficulty=all_trails.groupby('difficulty_rating')
data_by_difficulty=all_trails.groupby('difficulty_rating')[all_trails.columns].apply(lambda g: g.values.tolist()).to_dict()

In [ ]:
df_1=pd.DataFrame.from_dict(data_by_difficulty[1])
df_1.columns=all_trails.columns

df_3=pd.DataFrame.from_dict(data_by_difficulty[3])
df_3.columns=all_trails.columns

df_5=pd.DataFrame.from_dict(data_by_difficulty[5])
df_5.columns=all_trails.columns

df_7=pd.DataFrame.from_dict(data_by_difficulty[7])
df_7.columns=all_trails.columns


#### As a part of the UI, the user will be inputting this value, right now we are taking it as a prompt

In [ ]:
anvil.server.callable


In [ ]:
def prompt_for_integer(prompt):
    while True:
        try:
            val=int(input(prompt))
            if val not in [1,3,5,7]:
              print("wrong difficulty")
            else:
              return(val)
        except ValueError:
            print("Please enter a valid integer.")

diff_input = prompt_for_integer("Enter the difficulty: ")

Enter the difficulty: 3


In [ ]:
diff_input
df_final=pd.DataFrame()
if diff_input>=1:
  df_final=df_final.append(df_1)
if diff_input>=3:
  df_final=df_final.append(df_3)
if diff_input>=5:
  df_final=df_final.append(df_5)
if diff_input>=7:
  df_final=df_final.append(df_7) 
df_final

,Unnamed: 0,trail_id,name,area_name,city_name,state_name,country_name,popularity,length,elevation_gain,...,activity_skiing,activity_bike-touring,activity_whitewater-kayaking,activity_rails-trails,activity_ice-climbing,activity_surfing,activity_snowboarding,state_name_cat,city_name_cat,condition_cat
0,2,10267857,Exit Glacier Trail,Kenai Fjords National Park,Seward,Alaska,United States,17.7821,2896.812,81.9912,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,225,0
1,3,10236076,Horseshoe Lake Trail,Denali National Park,Denali National Park,Alaska,United States,16.2674,3379.614,119.7864,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,68,14
2,8,10236084,McKinley Station Trail,Denali National Park,Denali National Park,Alaska,United States,5.8885,4667.086,113.9952,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,68,14
3,10,10265252,McKinley Bar Trail,Denali National Park,Denali National Park,Alaska,United States,5.6536,7402.964,295.9608,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,68,8
4,14,10262541,Bartlett Cove Forest Loop Trail,Glacier Bay National Park,Gustavus,Alaska,United States,3.8939,1770.274,33.8328,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,98,6
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1448,3297,10000825,Byron Ledge Trail from Devastation Trailhead [...,Hawaii Volcanoes National Park,Pahala,Hawaii,United States,0.5862,7081.096,123.7488,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,7,191,12
1449,3298,10391957,Byron Ledge & Halema'uma'u Loop [CLOSED],Hawaii Volcanoes National Park,Volcano,Hawaii,United States,0.5015,5149.888,147.8280,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,7,263,12
1450,3299,10005997,Pipiwai Trail and Waimoku Falls,Haleakala National Park,Hana,Hawaii,United States,33.3147,5471.756,249.9360,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,7,99,12
1451,3305,10027793,Pa Ka'oao Trail,Haleakala National Park,Kula,Hawaii,United States,11.5722,965.604,40.8432,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,7,140,12


Step 5: Clustering the user custom difficulty data based on various features
#### Now we cluster the above dataframe based on various metrics like the activities, weather, fetaures of the trail etc.

In [ ]:
# clustering
data_pan=df_final[['trail_id','popularity', 'elevation_gain','condition_cat',
       'avg_rating','Number_of_Campgrounds', 'Rating', 'temp_c', 'wind_kph',
       'pressure_mb', 'precip_mm', 'humidity', 'cloud', 'feelslike_c',
       'vis_km', 'uv', 'gust_kph','length_miles', 'features_count', 'activities_count', 'feature_dogs-no',
       'feature_forest', 'feature_river', 'feature_views', 'feature_waterfall',
       'feature_wild-flowers', 'feature_wildlife', 'feature_partially-paved',
       'feature_lake', 'feature_kids', 'feature_historic-site',
       'feature_dogs-leash', 'feature_ada', 'feature_beach', 'feature_cave',
       'feature_paved', 'feature_strollers', 'feature_', 'feature_hot-springs',
       'feature_city-walk', 'feature_rails-trails', 'feature_dogs',
       'activity_birding', 'activity_camping', 'activity_hiking',
       'activity_nature-trips', 'activity_trail-running', 'activity_walking',
       'activity_fishing', 'activity_backpacking', 'activity_sea-kayaking',
       'activity_canoeing', 'activity_mountain-biking',
       'activity_horseback-riding', 'activity_scenic-driving',
       'activity_road-biking', 'activity_off-road-driving',
       'activity_rock-climbing', 'activity_snowshoeing',
       'activity_cross-country-skiing', 'activity_fly-fishing',
       'activity_paddle-sports', 'activity_skiing', 'activity_bike-touring',
       'activity_whitewater-kayaking', 'activity_rails-trails',
       'activity_ice-climbing', 'activity_surfing', 'activity_snowboarding']]
data_pan=data_pan.fillna(0)
data_pan

,trail_id,popularity,elevation_gain,condition_cat,avg_rating,Number_of_Campgrounds,Rating,temp_c,wind_kph,pressure_mb,...,activity_cross-country-skiing,activity_fly-fishing,activity_paddle-sports,activity_skiing,activity_bike-touring,activity_whitewater-kayaking,activity_rails-trails,activity_ice-climbing,activity_surfing,activity_snowboarding
0,10267857,17.7821,81.9912,0,4.5,1.0,4.8,5.0,20.2,1000.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,10236076,16.2674,119.7864,14,4.5,1.0,5.0,-9.0,9.0,1002.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,10236084,5.8885,113.9952,14,4.5,1.0,5.0,-9.0,9.0,1002.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,10265252,5.6536,295.9608,8,4.0,1.0,5.0,-7.3,6.8,1008.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,10262541,3.8939,33.8328,6,4.5,1.0,4.8,4.4,22.0,1003.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1448,10000825,0.5862,123.7488,12,3.5,1.0,4.6,18.4,10.4,1018.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1449,10391957,0.5015,147.8280,12,4.5,1.0,4.6,18.4,10.4,1018.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1450,10005997,33.3147,249.9360,12,5.0,0.0,0.0,20.5,10.1,1018.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1451,10027793,11.5722,40.8432,12,4.5,0.0,0.0,26.0,13.0,1017.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


We are currently creating 5 clusters, we tried with lower and higher numbers but the data was either getting too accumulated or too scattered. We are using elkan algorithm which is supposed to work with one of the best efficiencies.

In [ ]:
from sklearn.cluster import KMeans
num_cluster=5
kmeans = KMeans(n_clusters=num_cluster,algorithm='elkan')
kmeans.fit(data_pan)


KMeans(algorithm='elkan', n_clusters=5)

#### Now we generate the clusters amongst the data points generated from the above alogorithm to view the rows that fall within different clusters. These clusters are assigned to a dictionary for easy access and understanding.



In [ ]:
clusters = {}
labels=kmeans.labels_
n = 0
for item in labels:
  if item in clusters:
    clusters[item].append(data_pan.iloc[n])
  else:
    clusters[item] = [data_pan.iloc[n]]
    n +=1

#### As a next step, we are going to sort all the clusters based on the popularity. We are doing this as we want to display 5 top trails to the user and ask them for a 'Like' or a 'Dislike'. This will help us understand user choices better and we will be able to provide good recommendations.

In [ ]:
for i in range(0,5):
  clusters[i] = sorted(clusters[i],key=lambda x:x['popularity'],reverse=True)

#### We create a dataframe 'data_to_display' to get the top 5 elements of each cluster. This dataframe will be passed to the UI to be displayed to the User

In [ ]:
data_to_display=pd.DataFrame()
for i in range(0,5):
  #print(clusters[i][0])
  data_to_display=data_to_display.append((clusters[i][0]))
data_to_display=data_to_display.reset_index()
data_to_display=data_to_display.drop(['index'],axis=1)
data_to_display

,trail_id,popularity,elevation_gain,condition_cat,avg_rating,Number_of_Campgrounds,Rating,temp_c,wind_kph,pressure_mb,...,activity_cross-country-skiing,activity_fly-fishing,activity_paddle-sports,activity_skiing,activity_bike-touring,activity_whitewater-kayaking,activity_rails-trails,activity_ice-climbing,activity_surfing,activity_snowboarding
0,10236084.0,5.8885,113.9952,14.0,4.5,1.0,5.0,-9.0,9.0,1002.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,10236076.0,16.2674,119.7864,14.0,4.5,1.0,5.0,-9.0,9.0,1002.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,10267857.0,17.7821,81.9912,0.0,4.5,1.0,4.8,5.0,20.2,1000.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,10262541.0,3.8939,33.8328,6.0,4.5,1.0,4.8,4.4,22.0,1003.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,10265252.0,5.6536,295.9608,8.0,4.0,1.0,5.0,-7.3,6.8,1008.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [ ]:
df_new=df_final
df_new

,Unnamed: 0,trail_id,name,area_name,city_name,state_name,country_name,popularity,length,elevation_gain,...,activity_skiing,activity_bike-touring,activity_whitewater-kayaking,activity_rails-trails,activity_ice-climbing,activity_surfing,activity_snowboarding,state_name_cat,city_name_cat,condition_cat
0,2,10267857,Exit Glacier Trail,Kenai Fjords National Park,Seward,Alaska,United States,17.7821,2896.812,81.9912,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,225,0
1,3,10236076,Horseshoe Lake Trail,Denali National Park,Denali National Park,Alaska,United States,16.2674,3379.614,119.7864,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,68,14
2,8,10236084,McKinley Station Trail,Denali National Park,Denali National Park,Alaska,United States,5.8885,4667.086,113.9952,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,68,14
3,10,10265252,McKinley Bar Trail,Denali National Park,Denali National Park,Alaska,United States,5.6536,7402.964,295.9608,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,68,8
4,14,10262541,Bartlett Cove Forest Loop Trail,Glacier Bay National Park,Gustavus,Alaska,United States,3.8939,1770.274,33.8328,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,98,6
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1448,3297,10000825,Byron Ledge Trail from Devastation Trailhead [...,Hawaii Volcanoes National Park,Pahala,Hawaii,United States,0.5862,7081.096,123.7488,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,7,191,12
1449,3298,10391957,Byron Ledge & Halema'uma'u Loop [CLOSED],Hawaii Volcanoes National Park,Volcano,Hawaii,United States,0.5015,5149.888,147.8280,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,7,263,12
1450,3299,10005997,Pipiwai Trail and Waimoku Falls,Haleakala National Park,Hana,Hawaii,United States,33.3147,5471.756,249.9360,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,7,99,12
1451,3305,10027793,Pa Ka'oao Trail,Haleakala National Park,Kula,Hawaii,United States,11.5722,965.604,40.8432,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,7,140,12


In [ ]:
def prompt_for_hard_filters(prompt):
    while True:
        try:
            val=input(prompt)
            return(val)
        except ValueError:
            print("Please enter a valid value")

Step 6: Collecting reviews of user about the most popular trails displayed
#### The review of all the choices displayed to the user is selected. This will help us generate data about the general liking of the user.

In [ ]:
review_1=prompt_for_hard_filters("Review of 1st choice")
review_2=prompt_for_hard_filters("Review of 2st choice")
review_3=prompt_for_hard_filters("Review of 3st choice")
review_4=prompt_for_hard_filters("Review of 4st choice")
review_5=prompt_for_hard_filters("Review of 5st choice")


KeyboardInterrupt: ignored


Step 6: Now lets take the hard coded filters from the user to create a custom recommendation:
1. Location: This will be a city that the user will choose from a list of cities in the dropdown.
2. Radius: This is the radius in miles, within which the user wants to find a trail from the location selected above.



In [ ]:
data_hard_filter=pd.concat([df_new.set_index('trail_id'),all_trails.set_index('trail_id')], axis=1, join='inner').reset_index()
data_hard_filter = data_hard_filter.loc[:,~data_hard_filter.columns.duplicated()]
data_hard_filter

,trail_id,Unnamed: 0,name,area_name,city_name,state_name,country_name,popularity,length,elevation_gain,...,activity_skiing,activity_bike-touring,activity_whitewater-kayaking,activity_rails-trails,activity_ice-climbing,activity_surfing,activity_snowboarding,state_name_cat,city_name_cat,condition_cat
0,10267857,2,Exit Glacier Trail,Kenai Fjords National Park,Seward,Alaska,United States,17.7821,2896.812,81.9912,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,225,0
1,10236076,3,Horseshoe Lake Trail,Denali National Park,Denali National Park,Alaska,United States,16.2674,3379.614,119.7864,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,68,14
2,10236084,8,McKinley Station Trail,Denali National Park,Denali National Park,Alaska,United States,5.8885,4667.086,113.9952,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,68,14
3,10265252,10,McKinley Bar Trail,Denali National Park,Denali National Park,Alaska,United States,5.6536,7402.964,295.9608,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,68,8
4,10262541,14,Bartlett Cove Forest Loop Trail,Glacier Bay National Park,Gustavus,Alaska,United States,3.8939,1770.274,33.8328,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0,98,6
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2333,10000825,3297,Byron Ledge Trail from Devastation Trailhead [...,Hawaii Volcanoes National Park,Pahala,Hawaii,United States,0.5862,7081.096,123.7488,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,7,191,12
2334,10391957,3298,Byron Ledge & Halema'uma'u Loop [CLOSED],Hawaii Volcanoes National Park,Volcano,Hawaii,United States,0.5015,5149.888,147.8280,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,7,263,12
2335,10005997,3299,Pipiwai Trail and Waimoku Falls,Haleakala National Park,Hana,Hawaii,United States,33.3147,5471.756,249.9360,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,7,99,12
2336,10027793,3305,Pa Ka'oao Trail,Haleakala National Park,Kula,Hawaii,United States,11.5722,965.604,40.8432,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,7,140,12


In [ ]:
loca=prompt_for_hard_filters("City ")
dis=prompt_for_hard_filters("Radius in miles ")

City Chicago
Radius in miles 50


#### Using the geopy API, we get the longitute and latitude of the location given by the user. As a long term process, when we ahve the list of cities in the UI, we can store this information in the backend and might not need to call this API again and again.

In [ ]:
from geopy.geocoders import Nominatim
address=loca
geolocator = Nominatim(user_agent="Test")
location = geolocator.geocode(address)
print((location.latitude, location.longitude))

(41.8755616, -87.6244212)


In [ ]:
loc_object=(location.latitude, location.longitude)
loc_object

(41.8755616, -87.6244212)

#### The below function retrieves the distance between any two places using the haversine library

In [ ]:
from haversine import haversine, Unit
def getDistance(origin,des_lat,des_long):
  destination=(des_lat,des_long)
  return haversine(origin, destination, unit=Unit.MILES)

We create a column in the dataframe, which calculates the distance between the location chosen by the user and the different locations we have in our database. This will help us eliminate the places which are farther than the distance stated by the user. Moreover, while building this application at scale, we will be storing the information of the distance between the hikes and the cities in our database, so that we do not have to call thsi function again and again.

In [ ]:
data_hard_filter['distance_from_location']=data_hard_filter.apply (lambda row: getDistance(loc_object,row['lat'],row['lng']), axis=1)

We apply a hard filter on teh location and remove the trails that are farther away from the location chosen.

In [ ]:
data_hard_filter=data_hard_filter[data_hard_filter['distance_from_location']<=float(dis)]
data_hard_filter

,trail_id,Unnamed: 0,name,area_name,city_name,state_name,country_name,popularity,length,elevation_gain,...,activity_bike-touring,activity_whitewater-kayaking,activity_rails-trails,activity_ice-climbing,activity_surfing,activity_snowboarding,state_name_cat,city_name_cat,condition_cat,distance_from_location


Step 7: This file is being exported to be now used for collaborative filtering in another notebook

In [ ]:
from google.colab import drive
drive.mount('/content/drive',force_remount=True)
name_of_File='UserWith5difficulty_Chicago_Location_Within300Miles.csv'
data_hard_filter.to_csv(name_of_File)
!cp UserWith5difficulty_Chicago_Location_Within300Miles.csv "drive/MyDrive/243/Data/"

Mounted at /content/drive
cp: cannot create regular file 'drive/MyDrive/243/Data/': No such file or directory


### Step 8: Next Phase:
Now as a last phase, we will apply cosine similarity between the dataframe we have created above and the trails that were highly rated by the user. 

In [ ]:
cos_data=data_hard_filter[data_to_display.columns]
cos_data

,trail_id,popularity,elevation_gain,condition_cat,avg_rating,Number_of_Campgrounds,Rating,temp_c,wind_kph,pressure_mb,...,activity_cross-country-skiing,activity_fly-fishing,activity_paddle-sports,activity_skiing,activity_bike-touring,activity_whitewater-kayaking,activity_rails-trails,activity_ice-climbing,activity_surfing,activity_snowboarding
332,10033779,6.2936,43.8912,0,4.5,1.0,3.4,-2.9,0.0,1024.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
333,10323594,6.2548,28.9560,0,4.0,1.0,3.4,-2.9,0.0,1024.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
334,10338879,5.8174,40.8432,0,4.5,NaN,NaN,-2.9,0.0,1024.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
335,10351455,3.5550,5.7912,0,4.0,NaN,NaN,-2.2,0.0,1024.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
336,10331078,3.0381,40.8432,0,4.0,1.0,3.4,-2.9,0.0,1024.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
337,10496321,2.8118,20.7264,0,4.5,NaN,NaN,-2.0,0.0,1024.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
338,10495796,1.8294,32.9184,0,4.0,NaN,NaN,-2.0,0.0,1024.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
339,10337618,1.7891,13.7160,0,3.5,NaN,NaN,-2.0,0.0,1024.0,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
340,10369418,1.7808,1.8288,0,3.5,1.0,5.0,-2.0,0.0,1024.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
341,10496311,0.6880,4.8768,0,3.0,NaN,NaN,-2.0,0.0,1024.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


We replace the null values of Campgrounds and Rating as 0 as we do not have data of all the trails from google ratings

In [ ]:
cos_data['Number_of_Campgrounds']=cos_data['Number_of_Campgrounds'].fillna(0)
cos_data['Rating']=cos_data['Rating'].fillna(0)

We create a df which has only the trails that were reviewed by the user from the top 5 trails, and we select the trails whaich were rated 3 or above

In [ ]:
cos_dict={}
df=pd.DataFrame()
for i in range(1,6):
  if(int(locals()['review_'+str(i)])>=3):
    df=df.append(data_to_display.iloc[i-1])
df

,trail_id,popularity,elevation_gain,condition_cat,avg_rating,Number_of_Campgrounds,Rating,temp_c,wind_kph,pressure_mb,...,activity_cross-country-skiing,activity_fly-fishing,activity_paddle-sports,activity_skiing,activity_bike-touring,activity_whitewater-kayaking,activity_rails-trails,activity_ice-climbing,activity_surfing,activity_snowboarding
0,10267857.0,17.7821,81.9912,0.0,4.5,1.0,4.8,5.0,20.2,1000.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,10236084.0,5.8885,113.9952,14.0,4.5,1.0,5.0,-9.0,9.0,1002.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,10262541.0,3.8939,33.8328,6.0,4.5,1.0,4.8,4.4,22.0,1003.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,10236076.0,16.2674,119.7864,14.0,4.5,1.0,5.0,-9.0,9.0,1002.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


Using spacial, we identify the similarity between each value of the above dataframe and the one we generated after applying the filters

In [ ]:
from scipy import spatial
li=[]
for i in range(0,len(cos_data)):
  in_list=[]
  for j in range(0,len(df)):
    result =1-spatial.distance.cosine(cos_data.iloc[i], df.iloc[j])
    in_list.append(result)
  li.append(in_list)
print(li)

[[0.9999999999747909, 0.9999999999186073, 0.9999999999378592, 0.9999999999141626], [0.9999999999777524, 0.9999999999145878, 0.9999999999464452, 0.9999999999092344], [nan, nan, nan, nan], [nan, nan, nan, nan], [0.999999999982814, 0.9999999999235277, 0.9999999999464493, 0.9999999999184889], [nan, nan, nan, nan], [nan, nan, nan, nan], [nan, nan, nan, nan], [0.9999999999570934, 0.9999999998893329, 0.9999999999430933, 0.9999999998820154], [nan, nan, nan, nan], [nan, nan, nan, nan], [nan, nan, nan, nan], [nan, nan, nan, nan], [nan, nan, nan, nan], [0.9999999999805977, 0.9999999999354866, 0.9999999999271517, 0.9999999999332925], [0.999999999977757, 0.9999999999108196, 0.9999999999443566, 0.999999999905268], [0.9999999999853871, 0.9999999999301411, 0.9999999999436214, 0.9999999999260115], [nan, nan, nan, nan]]


We use the following code to generate the top 10 results and append the highly rated results of the user to this final dataframe that we will provide to the user


In [ ]:
final_recom=[]
for i in range(0,min(10,len(cos_data))):
  max_value, max_index = max((x, (i, j))
                              for i, row in enumerate(li)
                              for j, x in enumerate(row))
  print(max_value)
  print(max_index)
  final_recom.append(cos_data.iloc[max_index[0]])
  li[max_index[0]][max_index[1]]=0

df_final_recom=pd.DataFrame(final_recom)
df_final_recom=df_final_recom.append(df)
df_final_recom=df_final_recom.drop_duplicates()
df_final_recom=df_final_recom.reset_index().drop(['index'],axis=1)
df_final_recom

0.9999999999436214
(16, 2)
0.9999999999430933
(8, 2)
0.9999999999378592
(0, 2)
0.9999999999354866
(14, 1)
0.9999999999332925
(14, 3)
0.9999999999301411
(16, 1)
0.9999999999271517
(14, 2)
0.9999999999260115
(16, 3)
0.9999999999235277
(4, 1)
0.9999999999186073
(0, 1)


,trail_id,popularity,elevation_gain,condition_cat,avg_rating,Number_of_Campgrounds,Rating,temp_c,wind_kph,pressure_mb,...,activity_cross-country-skiing,activity_fly-fishing,activity_paddle-sports,activity_skiing,activity_bike-touring,activity_whitewater-kayaking,activity_rails-trails,activity_ice-climbing,activity_surfing,activity_snowboarding
0,10249715.0,5.4929,52.7304,0.0,4.0,1.0,3.4,-2.9,0.0,1024.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,10369418.0,1.7808,1.8288,0.0,3.5,1.0,5.0,-2.0,0.0,1024.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,10033779.0,6.2936,43.8912,0.0,4.5,1.0,3.4,-2.9,0.0,1024.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,10018164.0,8.6337,78.9432,0.0,4.0,1.0,3.4,-2.9,0.0,1024.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,10331078.0,3.0381,40.8432,0.0,4.0,1.0,3.4,-2.9,0.0,1024.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
5,10267857.0,17.7821,81.9912,0.0,4.5,1.0,4.8,5.0,20.2,1000.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
6,10236084.0,5.8885,113.9952,14.0,4.5,1.0,5.0,-9.0,9.0,1002.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
7,10262541.0,3.8939,33.8328,6.0,4.5,1.0,4.8,4.4,22.0,1003.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
8,10236076.0,16.2674,119.7864,14.0,4.5,1.0,5.0,-9.0,9.0,1002.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [ ]:
final_data_to_be_processed=pd.concat([df_final_recom.set_index('trail_id'),all_trails.set_index('trail_id')], axis=1, join='inner').reset_index()
final_data_to_be_processed = final_data_to_be_processed.loc[:,~final_data_to_be_processed.columns.duplicated()]
final_data_to_be_processed

,trail_id,popularity,elevation_gain,condition_cat,avg_rating,Number_of_Campgrounds,Rating,temp_c,wind_kph,pressure_mb,...,features,activities,units,lat,lng,condition,difficulty,length_inches,state_name_cat,city_name_cat
0,10249715.0,5.4929,52.7304,0.0,4.0,1.0,3.4,-2.9,0.0,1024.0,...,"['dogs-no', 'views', 'wild-flowers']","['birding', 'hiking', 'nature-trips', 'walking']",i,41.61477,-87.19301,Clear,moderate,5149.888,8,202
1,10369418.0,1.7808,1.8288,0.0,3.5,1.0,5.0,-2.0,0.0,1024.0,...,"['kids', 'views', 'wild-flowers', 'wildlife']","['birding', 'hiking', 'nature-trips', 'walking']",i,41.67481,-86.98662,Clear,easy,2253.076,8,165
2,10033779.0,6.2936,43.8912,0.0,4.5,1.0,3.4,-2.9,0.0,1024.0,...,"['beach', 'dogs-no', 'lake', 'kids', 'views']","['nature-trips', 'walking']",i,41.62209,-87.20799,Clear,easy,4345.218,8,202
3,10018164.0,8.6337,78.9432,0.0,4.0,1.0,3.4,-2.9,0.0,1024.0,...,"['ada', 'beach', 'dogs-no', 'lake', 'kids', 's...","['birding', 'hiking', 'nature-trips', 'trail-r...",i,41.62244,-87.20793,Clear,moderate,5471.756,8,202
4,10331078.0,3.0381,40.8432,0.0,4.0,1.0,3.4,-2.9,0.0,1024.0,...,"['forest', 'kids', 'river', 'views']","['hiking', 'nature-trips', 'trail-running', 'w...",i,41.61986,-87.17965,Clear,easy,3057.746,8,202
5,10267857.0,17.7821,81.9912,0.0,4.5,1.0,4.8,5.0,20.2,1000.0,...,"['dogs-no', 'partially-paved', 'views', 'wildl...","['hiking', 'walking']",i,60.18879,-149.63100,Clear,easy,2896.812,0,225
6,10236084.0,5.8885,113.9952,14.0,4.5,1.0,5.0,-9.0,9.0,1002.0,...,"['dogs-no', 'forest', 'kids', 'views', 'wild-f...","['birding', 'hiking', 'nature-trips', 'trail-r...",i,63.73324,-148.89659,Patchy light snow,easy,4667.086,0,68
7,10262541.0,3.8939,33.8328,6.0,4.5,1.0,4.8,4.4,22.0,1003.0,...,"['dogs-no', 'kids', 'views', 'wild-flowers', '...","['birding', 'hiking', 'nature-trips', 'walking...",i,58.45462,-135.88312,Light rain,easy,1770.274,0,98
8,10236076.0,16.2674,119.7864,14.0,4.5,1.0,5.0,-9.0,9.0,1002.0,...,"['dogs-no', 'forest', 'lake', 'kids', 'views',...","['birding', 'hiking', 'nature-trips', 'trail-r...",i,63.73661,-148.91500,Patchy light snow,easy,3379.614,0,68
